# Post-COVID E-Commerce Trends in Canada

## Introduction
### Purpose:
The COVID-19 pandemic significantly changed how Canadians shop, sell, and do business. E-commerce became essential for both consumers and businesses during lockdowns. Understanding this shift is important for three key reasons:
- Economic Impact
- Digital Equity and Access
- Policy and Future Planning
  
E-commerce is now a lasting part of Canada’s economy. By exploring how adoption changed across different regions, industries, and age groups, this project supports smarter investments in technology, innovation, and economic inclusion.

### Key Question:
How has e-commerce in Canada evolved across time, demographics, sectors, and regions since 2019, and what lasting changes emerged after the COVID-19 lockdowns?


## Background & Supplemental Research (incomplete)
- Contextual Literature or Survey Insights
    - Summarize findings from 3–5 key articles, reports, or policy briefs
    - Identify known trends, gaps, or conflicting views
- Timeline of Key Events
    - Dates of relevant events (e.g., lockdowns, policy shifts, major economic announcements)
- Definitions & Concepts
    - Clarify important terms (e.g., affordability stress, digital divide, retail e-commerce)
- Framing Hypotheses or Guiding Theories
    - Use social science, economics, or policy theory to guide your analysis


## Data Collection
### Primary Datasets
1. Statistics Canada. Table 33-10-0518-01  Percentage and average percentage of total sales made online in 2021, second quarter of 2022
2. Statistics Canada. Table 33-10-0646-01  Percentage and average percentage of total sales made online in 2022, first quarter of 2023
3. Statistics Canada. Table 33-10-0779-01  Percentage and average percentage of total sales made online in 2023
4. Statistics Canada. Table 33-10-0925-01  Percentage and average percentage of total sales made online in 2024
5. Statistics Canada. Table 22-10-0138-01  Goods and services purchased online by age group
6. Statistics Canada. Table 20-10-0056-03  Monthly retail trade e-commerce sales (x 1,000)
### Data Dictionary
####      Datasets 1-4: Percentage and Average Percentage of Total Sales Made Online in 2021-2024

| Variable Name       | Description                        | Type        | Units / Format               | Example Values           |
|---------------------|----------------------------------|-------------|-----------------------|--------------------------|
| Region              | Province or territory              | Categorical | String        | Alberta              |
| Business characterists| Attributes or features that define a business            | Categorical          | String          | Urban             |
| Percentage of total sales online | Bins from 0% to 100%| Numeric     | Percentage %             | 5.5                 |
| Average percentage  | Average Percentage of Total Sales Made Online | Numeric     | Percentage %           | 6.6                        |

####      Dataset 5: Goods and services purchased online by age group (2018 and 2020)

| Variable Name              | Description                                                    | Type               | Units / Format | Example Values       |
|---------------------------|----------------------------------------------------------------|--------------------|----------------|----------------------|
| Year                      | Year in which data was collected                               | Numeric (Discrete) | Year           | 2018                 |
| Online Goods Type         | Category of products sold online by businesses                 | Categorical        | String         | Physical goods       |
| Age Group                 | Age range of consumers (binned, from 15 to 65 years and older) | Numeric    | Percentage (%) | 29.1                 |
| Total, 15 Years and Over  | Consumers aged 15 and older                                    | Numeric            | Percentage (%) | 76.7                 |

####      Dataset 6: Monthly retail trade e-commerce sales
| Variable Name              | Description                                                    | Type               | Units / Format | Example Values       |
|---------------------------|----------------------------------------------------------------|--------------------|----------------|----------------------|
| Year/Month                      | Year and month in which data was collected                               | Numeric (Discrete) | Year-Month           | 2018-02                 |
| Industry         | category of the economy in which a business operates                | Categorical        | String         | Retail trade       |
| Sales                 | Type of sales          | Categorical   | String | Retail e-commerce sales               |
| Dollars (in thousands)  | Number of dollars spent in thousands                                   | Numeric            | Dollars ($) |          1110045.0      |


### Supplemental Datasets
- E.g., Census, survey data, time series indicators, external APIs


### Data Preparation
- Cleaning
    - Handling missing values, inconsistencies
- Merging / Joining
    - Integrating with supplemental datasets (e.g., by region or industry)
- Derived Variables
    - E.g., percent changes, category groupings, regional flags


In [17]:
import pandas as pd
import numpy as np
import os
import zipfile  

In [18]:

zip_path = '% Online Sales 2021.zip'
extract_path = 'unzipped_folder'  

# Create the folder
os.makedirs(extract_path, exist_ok=True)

# Extract the ZIP contents
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Remove any extracted files that have "meta" in the name
for file in os.listdir(extract_path):
    if 'meta' in file.lower():
        os.remove(os.path.join(extract_path, file))
        print(f"Deleted metadata file: {file}")

# List what's left
files = os.listdir(extract_path)
print(files)

Deleted metadata file: 33100518_MetaData.csv
['33100779.csv', '33100646.csv', '33100518.csv', '33100925.csv']


In [204]:
# Clean all data sets for each year
## % Online Sales 2021 by region and business characteristics
df21 = pd.read_csv('unzipped_folder/33100518.csv')
df21.rename(columns={'Percentage of total sales made online in 2021': 'Percentage of total sales online',
                    'GEO':'Region'}, inplace=True)
df21 = df21[['Region', 'Business characteristics', 'Percentage of total sales online', 'VALUE', 'STATUS']]
df21['Percentage of total sales online'] = df21['Percentage of total sales online'].str.replace(
    r'^.*?,\s*', '', regex=True)
df21['Percentage of total sales online'] = df21['Percentage of total sales online'].str.replace(
    'less than ', '', regex=False)

### pivot table
df21 = df21.pivot(index=['Region', 'Business characteristics'], columns='Percentage of total sales online', values='VALUE')

df21.rename(columns={'Average percentage of total sales made online in 2021':'Average percentage'}, inplace=True)

cols = list(df21.columns)
cols.remove('100%')
cols.remove('Average percentage')
cols.append('100%')
cols.append('Average percentage')

df21 = df21[cols]
df21 = df21.reset_index()

# Identify the percentage columns
percentage_cols = ['0%', '1% to 10%', '10% to 20%', '20% to 30%', '30% to 40%', 
                  '40% to 50%', '50% to 60%', '60% to 70%', '70% to 80%', 
                  '80% to 90%', '90% to 100%', '100%', 'Average percentage']

# Count missing values before filling
missing_before = df21[percentage_cols].isna().sum().sum()
print(f"Missing values before filling: {missing_before}")

# Fill missing values with zeros
df21[percentage_cols] = df21[percentage_cols].fillna(0)

# Count missing values after filling
missing_after = df21[percentage_cols].isna().sum().sum()
print(f"Missing values after filling with zeros: {missing_after}")

# Save the cleaned dataframe
df21.to_csv("df21.csv", index=False)

df21.head()

Missing values before filling: 703
Missing values after filling with zeros: 0


Percentage of total sales online,Region,Business characteristics,0%,1% to 10%,10% to 20%,20% to 30%,30% to 40%,40% to 50%,50% to 60%,60% to 70%,70% to 80%,80% to 90%,90% to 100%,100%,Average percentage
0,Alberta,1 to 4 employees,83.7,2.8,3.6,0.4,0.5,0.5,1.5,0.0,1.1,0.3,0.8,4.9,8.3
1,Alberta,100 or more employees,76.2,9.3,4.7,2.6,1.5,0.6,0.3,0.2,0.7,1.0,2.0,0.8,6.6
2,Alberta,20 to 99 employees,68.9,13.9,7.3,1.6,0.4,2.4,1.0,0.5,1.0,0.0,2.3,0.7,7.1
3,Alberta,5 to 19 employees,70.4,12.8,6.1,1.8,2.5,0.5,2.2,0.6,0.1,1.3,0.8,0.9,6.7
4,Alberta,Accommodation and food services [72],38.9,15.9,16.6,5.2,0.8,3.0,8.8,1.1,1.7,2.1,3.6,2.4,19.0


In [202]:
## % Online Sales 2022 by region and business characteristics
df22 = pd.read_csv('unzipped_folder/33100646.csv')
df22.rename(columns={'Percentage of total sales made online in 2022': 'Percentage of total sales online',
                    'GEO':'Region'}, inplace=True)
df22 = df22[['Region', 'Business characteristics', 'Percentage of total sales online', 'VALUE', 'STATUS']]
### pivot table
df22 = df22.pivot(index=['Region', 'Business characteristics'], columns='Percentage of total sales online', values='VALUE')

cols = list(df22.columns)
cols.remove('100%')
cols.remove('Average percentage')
cols.append('100%')
cols.append('Average percentage')

df22 = df22[cols]
df22 = df22.reset_index()

# Check the current missing values in the dataset
missing_before = df22.isna().sum().sum()
print(f"Total missing values before filling: {missing_before}")

# Identify the percentage columns (all columns except 'Region' and 'Business characteristics')
percentage_cols = [col for col in df22.columns if col not in ['Region', 'Business characteristics']]

# Count missing values in percentage columns before filling
missing_pct_before = df22[percentage_cols].isna().sum().sum()
print(f"Missing values in percentage columns before filling: {missing_pct_before}")

# Fill missing values with zeros in percentage columns
df22[percentage_cols] = df22[percentage_cols].fillna(0)

# Count missing values after filling
missing_after = df22.isna().sum().sum()
print(f"Total missing values after filling with zeros: {missing_after}")

# Save the cleaned dataframe
df22.to_csv("df22.csv", index=False)

# Display the first few rows of the cleaned dataframe
print("\nFirst few rows of the cleaned dataframe:")

df22.head()

Total missing values before filling: 790
Missing values in percentage columns before filling: 790
Total missing values after filling with zeros: 0

First few rows of the cleaned dataframe:


Percentage of total sales online,Region,Business characteristics,0%,1% to 9%,10% to 19%,20% to 29%,30% to 39%,40% to 49%,50% to 59%,60% to 69%,70% to 79%,80% to 89%,90% to 99%,100%,Average percentage
0,Alberta,1 to 4 employees,87.5,3.4,1.5,1.6,0.3,0.6,1.0,0.0,0.3,1.9,0.7,1.4,5.3
1,Alberta,100 or more employees,57.4,25.1,6.0,3.7,0.1,0.8,0.5,0.0,0.4,2.1,0.0,4.0,9.1
2,Alberta,20 to 99 employees,75.4,11.0,5.4,0.3,0.2,0.1,1.7,3.6,0.9,0.7,0.3,0.3,6.2
3,Alberta,5 to 19 employees,77.1,8.5,4.5,2.2,3.5,0.5,0.4,1.4,1.1,0.1,0.1,0.8,5.3
4,Alberta,Accommodation and food services [72],38.2,12.9,6.8,5.9,11.4,6.1,2.0,11.3,2.1,0.4,1.8,1.1,21.5


In [201]:
## % Online Sales 2023 by region and business characteristics
df23 = pd.read_csv('unzipped_folder/33100779.csv')
df23.rename(columns={'Percentage of total sales made online in 2023': 'Percentage of total sales online',
                    'GEO':'Region'}, inplace=True)
df23 = df23[['Region', 'Business characteristics', 'Percentage of total sales online', 'VALUE', 'STATUS']]
### pivot table
df23 = df23.pivot(index=['Region', 'Business characteristics'], columns='Percentage of total sales online', values='VALUE')

df23.rename(columns={'Percentage of total sales made online in 2023, average percentage':'Average percentage'}, inplace=True)

cols = list(df23.columns)
cols.remove('100%')
cols.remove('Average percentage')
cols.append('100%')
cols.append('Average percentage')

df23 = df23[cols]
df23 = df23.reset_index()

# Check the current missing values in the dataset
missing_before = df23.isna().sum().sum()
print(f"Total missing values before filling: {missing_before}")

# Identify the percentage columns (all columns except 'Region' and 'Business characteristics')
percentage_cols = [col for col in df23.columns if col not in ['Region', 'Business characteristics']]

# Count missing values in percentage columns before filling
missing_pct_before = df23[percentage_cols].isna().sum().sum()
print(f"Missing values in percentage columns before filling: {missing_pct_before}")

# Fill missing values with zeros in percentage columns
df23[percentage_cols] = df23[percentage_cols].fillna(0)

# Count missing values after filling
missing_after = df23.isna().sum().sum()
print(f"Total missing values after filling with zeros: {missing_after}")

# Save the cleaned dataframe
df23.to_csv("df23.csv", index=False)

# Display the first few rows of the cleaned dataframe
print("\nFirst few rows of the cleaned dataframe:")
df23.head()

Total missing values before filling: 798
Missing values in percentage columns before filling: 798
Total missing values after filling with zeros: 0

First few rows of the cleaned dataframe:


Percentage of total sales online,Region,Business characteristics,0%,1% to 9%,10% to 19%,20% to 29%,30% to 39%,40% to 49%,50% to 59%,60% to 69%,70% to 79%,80% to 89%,90% to 99%,100%,Average percentage
0,Alberta,1 to 4 employees,83.6,3.7,3.3,1.4,1.2,0.7,1.5,0.1,0.0,0.5,2.6,1.4,6.5
1,Alberta,100 or more employees,84.6,3.9,9.0,0.8,0.0,0.2,0.0,0.7,0.4,0.2,0.1,0.0,2.4
2,Alberta,20 to 99 employees,67.5,10.7,5.0,5.3,0.5,1.5,1.4,0.2,3.6,0.2,2.6,1.6,10.4
3,Alberta,5 to 19 employees,82.7,8.1,0.6,0.7,3.7,0.6,0.3,0.2,0.7,0.8,0.4,1.3,5.0
4,Alberta,Accommodation and food services [72],58.3,9.6,3.6,12.0,8.3,5.2,0.4,0.3,0.5,0.6,1.1,0.0,10.6


In [199]:
## % Online Sales 2024 by region and business characteristics
df24 = pd.read_csv('unzipped_folder/33100925.csv')
df24.rename(columns={'Percentage of total sales made online in 2024': 'Percentage of total sales online',
                    'GEO':'Region'}, inplace=True)
df24 = df24[['Region', 'Business characteristics', 'Percentage of total sales online', 'VALUE', 'STATUS']]
df24['Percentage of total sales online'] = df24['Percentage of total sales online'].str.replace(
    r'^.*?,\s*', '', regex=True)
df24['Percentage of total sales online'] = df24['Percentage of total sales online'].str.replace(
    'less than ', '', regex=False)
### pivot table
df24 = df24.pivot(index=['Region', 'Business characteristics'], columns='Percentage of total sales online', values='VALUE')

df24.rename(columns={'average percentage':'Average percentage'}, inplace=True)

cols = list(df24.columns)
cols.remove('100%')
cols.remove('Average percentage')
cols.append('100%')
cols.append('Average percentage')

df24 = df24[cols]
df24 = df24.reset_index()

# Check the current missing values in the dataset
missing_before = df24.isna().sum().sum()
print(f"Total missing values before filling: {missing_before}")

# Identify the percentage columns (all columns except 'Region' and 'Business characteristics')
percentage_cols = [col for col in df24.columns if col not in ['Region', 'Business characteristics']]

# Count missing values in percentage columns before filling
missing_pct_before = df24[percentage_cols].isna().sum().sum()
print(f"Missing values in percentage columns before filling: {missing_pct_before}")

# Fill missing values with zeros in percentage columns
df24[percentage_cols] = df24[percentage_cols].fillna(0)

# Count missing values after filling
missing_after = df24.isna().sum().sum()
print(f"Total missing values after filling with zeros: {missing_after}")

# Save the cleaned dataframe
df24.to_csv("df24.csv", index=False)

# Display the first few rows of the cleaned dataframe
print("\nFirst few rows of the cleaned dataframe:")
df24.head()

Total missing values before filling: 920
Missing values in percentage columns before filling: 920
Total missing values after filling with zeros: 0

First few rows of the cleaned dataframe:


Percentage of total sales online,Region,Business characteristics,0%,1% to 9%,10% to 19%,20% to 29%,30% to 39%,40% to 49%,50% to 59%,60% to 69%,70% to 79%,80% to 89%,90% to 99%,100%,Average percentage
0,Alberta,1 to 4 employees,88.0,3.2,3.3,1.1,0.3,0.1,0.4,0.0,0.0,0.5,0.9,2.3,4.7
1,Alberta,100 or more employees,73.6,11.3,9.4,0.9,2.7,0.0,0.3,0.0,0.0,1.0,0.6,0.3,4.6
2,Alberta,20 to 99 employees,77.0,14.4,4.4,0.7,1.0,0.2,0.6,0.2,0.1,0.7,0.8,0.0,3.9
3,Alberta,5 to 19 employees,77.7,10.9,4.4,1.1,2.8,0.2,1.5,0.3,0.0,0.5,0.3,0.2,4.0
4,Alberta,Accommodation and food services [72],52.1,18.3,12.3,7.0,5.4,1.2,0.7,0.2,0.3,0.0,2.5,0.0,9.4


In [128]:
zip_path = 'Goods and services purchased online by age group (2018 and 2020).zip'
extract_path = 'AgeGroup_folder'  

# Create the folder
os.makedirs(extract_path, exist_ok=True)

# Extract the ZIP contents
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Remove any extracted files that have "meta" in the name
for file in os.listdir(extract_path):
    if 'meta' in file.lower():
        os.remove(os.path.join(extract_path, file))
        print(f"Deleted metadata file: {file}")

# List what's left
files = os.listdir(extract_path)
print(files)

Deleted metadata file: 22100138_MetaData.csv
['22100138.csv']


In [210]:
# Goods and services purchased online by age group in Canada
df_age_group = pd.read_csv('AgeGroup_folder/22100138.csv')

df_age_group = df_age_group[['REF_DATE', 'Online shoppers and type of good and service', 'Age group', 'VALUE']]

df_age_group = df_age_group.rename(columns={'REF_DATE':'Year',
                                            'Online shoppers and type of good and service': 'Online Goods Type'})
df_age_group = df_age_group.pivot(index=['Year', 'Online Goods Type'], columns='Age group', values='VALUE')

df_age_group = df_age_group.reset_index()

missing_before = df_age_group.isna().sum().sum()
print(f"Total missing values before handling: {missing_before}")

df_age_group = df_age_group.drop([3, 10]) 

missing_after = df_age_group.isna().sum().sum()
print(f"Total missing values after handling: {missing_after}")

df_age_group.to_csv("df_age_group.csv", index=False)

df_age_group.head()

Total missing values before handling: 5
Total missing values after handling: 0


Age group,Year,Online Goods Type,15 to 24 years,25 to 44 years,45 to 64 years,65 years and over,"Total, 15 years and over"
0,2018,Digital goods or services,68.4,70.2,46.0,21.7,51.8
1,2018,Online shoppers,86.8,92.0,77.7,44.7,76.7
2,2018,Other services,55.8,75.8,58.1,29.1,57.4
4,2018,Peer-to-peer ride services,25.3,29.1,13.7,2.9,18.0
5,2018,Physical goods,73.9,84.1,66.9,34.1,66.5


In [144]:

zip_path = 'Monthly retail trade e-commerce sales.zip'
extract_path = 'MonthlyE-Sales_folder'  

# Create the folder
os.makedirs(extract_path, exist_ok=True)

# Extract the ZIP contents
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Remove any extracted files that have "meta" in the name
for file in os.listdir(extract_path):
    if 'meta' in file.lower():
        os.remove(os.path.join(extract_path, file))
        print(f"Deleted metadata file: {file}")

# List what's left
files = os.listdir(extract_path)
print(files)

Deleted metadata file: 20100056_MetaData.csv
['20100056.csv']


In [212]:
# Monthly E-commerce sales in canada by the dollar in thousands
df_Monthly_ESales = pd.read_csv('MonthlyE-Sales_folder/20100056.csv')
df_Monthly_ESales = df_Monthly_ESales[['REF_DATE', 'North American Industry Classification System (NAICS)', 'Sales', 'Adjustments','VALUE']]
df_Monthly_ESales = df_Monthly_ESales.rename(columns= {'REF_DATE':'Year/Month',
                                                       'North American Industry Classification System (NAICS)': 'Industry',
                                                       'VALUE':'Dollars (in thousands)'})


df_Monthly_ESales.to_csv("df_Monthly_ESales.csv", index=False)

missing_before = df_Monthly_ESales.isna().sum().sum()
print(f"Total missing values before handling: {missing_before}")
                     
df_Monthly_ESales.head()

Total missing values before handling: 9226


,Year/Month,Industry,Sales,Adjustments,Dollars (in thousands)
0,2017-01,Retail trade [44-45],Total retail sales,Unadjusted,41377009.0
1,2017-01,Retail trade [44-45],Total retail sales,Seasonally adjusted,50417235.0
2,2017-01,Retail trade [44-45],Retail e-commerce sales,Unadjusted,1110045.0
3,2017-01,Retail trade [44-45],Retail e-commerce sales,Seasonally adjusted,1236885.0
4,2017-01,Motor vehicle and parts dealers [441],Total retail sales,Unadjusted,10162441.0
